In [ ]:
import json
import requests
import time

def call_local_llm(url, role, prompt):
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }

    params = {
        "message_self": True  # 設定傳入的訊息參數
    }

    data = [
        {"role": "system", "content": role},
        {"role": "user", "content": prompt},
        # {"role": "assistant", "content": prompt}
    ]

    response = requests.post(url, headers=headers, params=params, json=data)
    return response.text
# 讀取 JSON 文件
with open('dataset/preliminary/questions_example.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 定義改寫問題的函數
def enhance_question(original_question: str, context: str = "", focus: str = "一般"):
    prompt_template = f"""
    請協助優化並生成多重搜尋問題，並根據以下原始問題：
    
    原始問題：{original_question}

    """
    
    system_prompt = """你是金融領域的專業問題優化助理，你擅長生成多重查詢的問題，
    根據輸入的一個問題，生成多個相關問題，並遵守以下規則：
    1. 只需直接輸出生成的問題，不提供額外解釋、文字及回答
    2. 保持問題簡潔（50字內）
    3. 確保包含完整的時間、主體、事件資訊
    4. 使用準確的金融專業術語
    5. 避免過度延伸原意、冗長敘述
    6. 最多生成4個相關問題
    """

    response = call_local_llm(
        url='http://127.0.0.1:8087/chat',
        role=system_prompt,
        prompt=prompt_template
    )
    
    return response

# 改寫所有問題並將改寫後的問題與原來的問題相加


for question in data['questions']:
    if 'query' in question:
        start_time = time.time()
        original_query = question['query']
        enhanced_query = enhance_question(original_query)
        print(f"原始問題：{original_query}")
        print(f"改寫後問題：{enhanced_query}")
        question['query'] = original_query + " " + enhanced_query
        end_time = time.time()
        print(f"改寫問題所需時間: {end_time - start_time} 秒")


# 將改寫後的問題寫入新的 JSON 文件
with open('enhanced_questions.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print("問題改寫完成並已輸出至 enhanced_questions.json")

In [1]:
from transformers import (
   BertTokenizerFast,
   AutoModelForMaskedLM,
   AutoModelForCausalLM,
   AutoModelForTokenClassification,
)

tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
model = AutoModelForTokenClassification.from_pretrained('ckiplab/bert-base-chinese')

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

c:\Users\marks\anaconda3\envs\poetry3.11\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\marks\.cache\huggingface\hub\models--ckiplab--bert-base-chinese. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/409M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ckiplab/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/409M [00:00<?, ?B/s]

In [21]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

ws_driver  = CkipWordSegmenter(model="bert-base")

ner_driver = CkipNerChunker(model="bert-base")

In [52]:
text = [
 """三三、 關係人交易
智邦公司及子公司（係智邦公司之關係人）間之交易、帳戶餘額、
收益及費損於合併時全數予以銷除，故未揭露於本附註。本公司與其
他關係人間之交易如下：
(一) 關係人名稱及其關係
關 係 人 名 稱 與 本 公 司 之 關 係
諾雲信息系統（上海）有限公司 實質關係人
智宇生醫股份有限公司 關聯企業
智上科技股份有限公司 子公司（註）
智采科技股份有限公司 關聯企業
註：智上科技股份有限公司於 112 年 4 月由關聯企業轉列為子公司。
(二) 營業收入
112年7月1日 111年7月1日 112年1月1日 111年1月1日
帳 列 項 目 關 係 人 名 稱 至9月30日 至9月30日 至9月30日 至9月30日
營業收入 智宇生醫公司 $ 104 $ 1,242 $ 1,297 $ 2,222
智上公司 - - 580 37
智采公司 - - ( 1,883 ) -
$ 104 $ 1,242 ( $ 6 ) $ 2,259
本公司對關係人銷貨之價格，係依據雙方議定之條件為之，並
無適當交易對象可資比較。
(三) 營業費用
112年7月1日 111年7月1日 112年1月1日 111年1月1日
帳 列 項 目 關 係 人 名 稱 至9月30日 至9月30日 至9月30日 至9月30日
營業費用 智采公司 $ 2,333 $ - $ 4,717 $ -
智上公司 - - 957 -
$ 2,333 $ - $ 5,674 $ -
本公司與關係人之交易，係依雙方議定條件決定。
(四) 營業外收入及損失
112年7月1日 111年7月1日 112年1月1日 111年1月1日
帳 列 項 目 關 係 人 名 稱 至9月30日 至9月30日 至9月30日 至9月30日
其他收入 智宇生醫公司 $ 23 $ 23 $ 69 $ 69
智上公司 - 70 - 221
智采公司 - - ( 24 ) -
$ 23 $ 93 $ 45 $ 290
本公司與關係人之交易，係依雙方議定條件決定。
- 45 -(五) 應收關係人款項
帳 列 項 目 關 係 人 名 稱 112年9月30日 111年12月31日 111年9月30日
應收帳款－關係人 智宇生醫公司 $ 37 $ 1,004 $ 167
智上公司 - 139 39
$ 37 $ 1,143 $ 206
本公司對關係人收款條件為交貨後 75 天至 90 天。
(六) 其他應收關係人款項
帳 列 項 目 關 係 人 名 稱 112年9月30日 111年12月31日 111年9月30日
其他應收款－關係人 諾雲信息公司 $ 1,371 $ 5,314 $ 5,389
智宇生醫公司 16 16 16
智上公司 - 3,628 3,695
智采公司 - 265 -
$ 1,387 $ 9,223 $ 9,100
本公司與關係人之交易，係依雙方議定條件決定。
(七) 預付款項
關 係 人 類 別 ／ 名 稱 112年9月30日 111年12月31日 111年9月30日
智上公司 $ - $ - $ 2,144
(八) 主要管理階層薪酬
112年7月1日 111年7月1日 112年1月1日 111年1月1日
至9月30日 至9月30日 至9月30日 至9月30日
短期員工福利 $ 38,077 $ 35,546 $ 168,962 $ 149,757
退職後福利 289 270 884 790
$ 38,366 $ 35,816 $ 169,846 $ 150,547
董事及其他主要管理階層之薪酬係由薪酬委員會依照個人績效
及市場趨勢決定。
三四、 質抵押之資產
本公司下列資產已提供作為關稅及履約保證等之擔保品：
112年9月30日 111年12月31日 111年9月30日
質押定期存款（帳列其他金
融資產－非流動） $ 79,623 $ 79,419 $ 79,419
- 46 -""",

   
]


ws  = ws_driver(text)
ner = ner_driver(text)

Inference: 100%|██████████| 1/1 [00:01<00:00,  1.28s/it]


In [53]:
ner

[[NerToken(word='諾雲信息公司', ner='ORG', idx=(1097, 1103)),
  NerToken(word='\n112年9月30日', ner='DATE', idx=(1555, 1565)),
  NerToken(word='111年12月31日', ner='DATE', idx=(1566, 1576)),
  NerToken(word='111年9月30日', ner='DATE', idx=(1577, 1586))]]